# Part 1: Getting current routes

Now, we get the current routes for each airport based on their wikipedia article based on a series of scripts developed in the first notebook. Save to a current_routes.csv file. 6


defining key functions

In [ ]:
#function to extract text between two found substrings
def extract_between(text, start, end):
    start_idx = text.find(start)
    if start_idx == -1:
        return "" #return empty string
    start_idx += len(start)
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return ""
    return text[start_idx:end_idx]
#function to get iata code from wikipedia name
def get_iata_code(iata,ref_data=pd.read_csv("./data/top_airports_basic_data.csv")):
    matched = ref_data[ref_data["wiki_name"]==iata].iloc[0]
    return matched["iata"]
    

def get_destinations(iata_source, wiki_name, file_append):
    search_url = f"https://en.wikipedia.org/w/index.php?title={wiki_name}&action=raw"
    response = requests.get(search_url)
    full_text = response.text
    #get to the passenger destinations section marked by a predictiable substrings ()
    # Call both and choose the first non-None result
    destinations_text = extract_between(full_text, "=== Passenger ===", "==") or extract_between(full_text, "===Passenger===", "==")
    destinations_list = destinations_text.split("<!-- -->")#split predictably
    
    return_list = [] #return a list of destination of tuple of the form (iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
    #ignore the first part, since that is some disclaimer
    for i in range(1, len(destinations_list)):
        airlineDest = destinations_list[i]#we get a list relating to airlines and their list of destinations
        #get information before and the seasonal destination section (if present)
        beforeSeasonal = ""
        afterSeasonal = None
        if "'''Seasonal:'''" in airlineDest:
            seasonalSplit = airlineDest.split("'''Seasonal:'''")
            beforeSeasonal = seasonalSplit[0]
            afterSeasonal = seasonalSplit[1]
        else: 
            beforeSeasonal = airlineDest 
        
        #get the airline based on a predictable pattern (in the first element)
        beforeSeasonalsplit = beforeSeasonal.split(",") #split based on commas
        airline_firstdest = beforeSeasonalsplit[0].split(" | [[")  #split between the airline and first destination
        airline = str(airline_firstdest[0]) #extract the airline
        airline = extract_between(airline,"[[","]]") #remove the [[]]
        #do a final split just in case (even if | is not found), [0] selects the current string
        firstdest = ""  #get first destination (assuming not seasonal) 
        if len(airline_firstdest) == 2:
            destination = airline_firstdest[1].split("|")[0].strip()
            destination_wiki_name = destination.replace(" ", "_") #replace destination to get the destination iata_code
            return_list.append((iata_source,wiki_name,destination_wiki_name,airline,False )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop thorugh the rest of the comma list for the before seasonal split if possible 
        for i in range(1, len(beforeSeasonalsplit)):
            destination = beforeSeasonalsplit[i]
            extracted_destination = extract_between(destination,"[[","]]")
            if (extracted_destination!=""): #assuming 
                destination_wiki_name = extracted_destination.split("|")[0].strip()
                destination_wiki_name = destination_wiki_name.replace(" ", "_")
                return_list.append((iata_source,wiki_name,destination_wiki_name,airline,False )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop through the seasonal destinations list if the current airline has it
        if (afterSeasonal):
            afterSeasonalSplit = afterSeasonal.split(",")
            for i in range(0, len(afterSeasonalSplit)):
                destination = afterSeasonalSplit[i]
                extracted_destination = extract_between(destination,"[[","]]")
                if (extracted_destination!=""): #assuming 
                    destination_wiki_name = extracted_destination.split("|")[0].strip()
                    destination_wiki_name = destination_wiki_name.replace(" ", "_")
                    return_list.append((iata_source,wiki_name,destination_wiki_name,airline,True )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
            
    return return_list

Start the file